In [ ]:
%matplotlib inline
# Load all necessary packages
import numpy as np


from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression

## Equalized odds post-processing 
# Odds equalizing post-processing algorithm
from aif360.algorithms.postprocessing import EqOddsPostprocessing

## Calbirating equalized odds post-processing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

## Load data set
### p 159

In [ ]:
compas_data  = load_preproc_data_compas()
priv_group   = [{'race': 1}]
unpriv_group = [{'race': 0}]    

In [ ]:
compas_data = load_preproc_data_compas()
compas_data

#### Divide dataset into train, validation, and test partitions 

In [ ]:
np.random.seed(132)
def split_data_trn_vld_tst(data_raw):
    dset_raw_trn, dset_raw_vt  = data_raw.split([0.7], 
                                               shuffle = True)
    dset_raw_vld, dset_raw_tst = dset_raw_vt.split([0.5], 
                                                   shuffle = True)
    
    return dset_raw_trn, dset_raw_vld, dset_raw_tst

In [ ]:
dset_raw_trn, dset_raw_vld, dset_raw_tst = split_data_trn_vld_tst(compas_data)

#### Metric for the original datasets (without any classifiers)

In [ ]:
## p 161
metric_orig_test = BinaryLabelDatasetMetric(dset_raw_tst, 
                             unprivileged_groups = unpriv_group,
                             privileged_groups   = priv_group)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % 
      metric_orig_test.mean_difference()) 

## Train classifier (logistic regression) on original training data
### not shown in book

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

dataset_orig_train_pred = dset_raw_trn.copy(deepcopy = True)
dataset_orig_valid_pred = dset_raw_vld.copy(deepcopy = True)
dataset_orig_test_pred  = dset_raw_tst.copy(deepcopy = True)

# Logistic regression classifier and predictions for training data
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dset_raw_trn.features)
y_train = dset_raw_trn.labels.ravel()
lmod = LogisticRegression()
lmod.fit(X_train, y_train)

fav_idx = np.where(lmod.classes_ == dset_raw_trn.favorable_label)[0][0]
y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

# Prediction probs for validation and testing data
X_valid = scale_orig.transform(dset_raw_vld.features)
y_valid_pred_prob = lmod.predict_proba(X_valid)[:,fav_idx]

X_test = scale_orig.transform(dset_raw_tst.features)
y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5
dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)
dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1,1)
dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred

y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
dataset_orig_valid_pred.labels = y_valid_pred
    
y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

#### Results before post-processing

In [ ]:
logit_classifier_metric = ClassificationMetric(dset_raw_tst, 
                                               dataset_orig_test_pred,
                                               unprivileged_groups = unpriv_group,
                                               privileged_groups   = priv_group)

print("Difference in GFPR between unprivileged and privileged groups")
print(logit_classifier_metric.difference(logit_classifier_metric.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(logit_classifier_metric.difference(logit_classifier_metric.generalized_false_negative_rate))
print("Mean difference in outcomes")
print(logit_classifier_metric.mean_difference()) 

### Note that logistic regression produces far more disparate outcome than what is found in raw data outcome!!

## Equality of Odds post-processing
### p 161

In [ ]:
SEED = 94

# Learn parameters to equalize odds and apply to create a new dataset
eop = EqOddsPostprocessing(unprivileged_groups = unpriv_group, 
                           privileged_groups   = priv_group, 
                           seed                = SEED)
eop = eop.fit(dset_raw_vld, dataset_orig_valid_pred)

eop_trn_pred = eop.predict(dataset_orig_train_pred)

eop_trn_metric = ClassificationMetric(dset_raw_trn, eop_trn_pred,
                             unprivileged_groups = unpriv_group,
                             privileged_groups  = priv_group)

print("TRAINING DATA")
print("Difference in GFPR between unprivileged and privileged groups")
print(eop_trn_metric.difference(eop_trn_metric.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(eop_trn_metric.difference(eop_trn_metric.generalized_false_negative_rate))
print("Mean difference in outcomes")
print(eop_trn_metric.mean_difference())

In [ ]:
eop_test_pred = eop.predict(dataset_orig_test_pred)
eop_test_metric = ClassificationMetric(dset_raw_tst, eop_test_pred,
                             unprivileged_groups = unpriv_group,
                             privileged_groups   = priv_group)
print("TEST DATA")
print("Difference in GFPR between unprivileged and privileged groups")
print(eop_test_metric.difference(eop_test_metric.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(eop_test_metric.difference(eop_test_metric.generalized_false_negative_rate))
print("Mean difference in outcomes")
print(eop_test_metric.mean_difference())

## Calibration preserving equalized odds
### p 166

In [ ]:
## p 166

# Learn parameters to equalize odds and apply to create a new dataset
cpp = CalibratedEqOddsPostprocessing(privileged_groups   = priv_group,
                                     unprivileged_groups = unpriv_group,
                                     cost_constraint     = 'fpr',
                                     seed                = 76)
cpp = cpp.fit(dset_raw_vld, dataset_orig_valid_pred)


dataset_transf_test_pred = cpp.predict(dataset_orig_test_pred)
cm_transf_test_metric = ClassificationMetric(dset_raw_tst, dataset_transf_test_pred,
                             unprivileged_groups = unpriv_group,
                             privileged_groups   = priv_group)


dataset_transf_train_pred = cpp.predict(dataset_orig_train_pred)
cm_transf_train_metric = ClassificationMetric(dset_raw_trn, dataset_transf_train_pred,
                             unprivileged_groups = unpriv_group,
                             privileged_groups   = priv_group)

print("CALIBRATING EQUALIZED ODDS PERFORMANCE ON TRAIN DATA")
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_transf_train_metric.difference(cm_transf_train_metric.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_transf_train_metric.difference(cm_transf_train_metric.generalized_false_negative_rate))
print("Mean difference in outcomes")
print(cm_transf_train_metric.mean_difference())
print("")
print("CALIBRATING EQUALIZED ODDS PERFORMANCE ON TEST DATA")
print("Difference in GFPR between unprivileged and privileged groups")
print(cm_transf_test_metric.difference(cm_transf_test_metric.generalized_false_positive_rate))
print("Difference in GFNR between unprivileged and privileged groups")
print(cm_transf_test_metric.difference(cm_transf_test_metric.generalized_false_negative_rate))
print("Mean difference in outcomes")
print(cm_transf_test_metric.mean_difference())